# [text gen] Domain related Text Generation with Azure OpenAI

This sample demonstrates how to generate a plain text file to train custom speech model.

> ✨ **_Note_** <br>
> Please check the custom speech support for each language before you get started - https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=stt#:~:text=Custom%20speech%20support

## Prerequisites

Git clone the repository to your local machine.

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

-   A subscription key for the Speech service. See [Try the speech service for free](https://docs.microsoft.com/azure/cognitive-services/speech-service/get-started).
-   Python 3.5 or later needs to be installed. Downloads are available [here](https://www.python.org/downloads/).
-   The Python Speech SDK package is available for Windows (x64 or x86) and Linux (x64; Ubuntu 16.04 or Ubuntu 18.04).
-   On Ubuntu 16.04 or 18.04, run the following commands for the installation of required packages:
    ```sh
    sudo apt-get update
    sudo apt-get install libssl3 libasound2
    ```
-   On Debian 9, run the following commands for the installation of required packages:
    ```sh
    sudo apt-get update
    sudo apt-get install libssl1.0.2 libasound2
    ```
-   On Windows you need the [Microsoft Visual C++ Redistributable for Visual Studio 2017](https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads) for your platform.

Configure a Python virtual environment for 3.10 or later:

1.  open the Command Palette (Ctrl+Shift+P).
1.  Search for Python: Create Environment.
1.  select Venv / Conda and choose where to create the new environment.
1.  Select the Python interpreter version. Create with version 3.10 or later.

```bash
python3 -m venv your_env_name
source your_env_name/bin/activate
pip install -r requirements.txt
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.


## 1. Generate synthetic dataset with Azure OpenAI


### Set up the environment variables


In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential

load_dotenv()

SPEECH_KEY = os.getenv("AZURE_AI_SPEECH_API_KEY")
SPEECH_REGION = os.getenv("AZURE_AI_SPEECH_REGION")
CUSTOM_SPEECH_LANG = os.getenv("CUSTOM_SPEECH_LANG")
CUSTOM_SPEECH_LOCALE = os.getenv("CUSTOM_SPEECH_LOCALE")

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if "/models" in aoai_api_endpoint:
    aoai_api_endpoint = aoai_api_endpoint.split("/models")[0] + "/models"

print("=== Azure AI Speech Info ===")
print(f"SPEECH_REGION={SPEECH_REGION}")
print(f"CUSTOM_SPEECH_LANG={CUSTOM_SPEECH_LANG}")
print(f"CUSTOM_SPEECH_LOCALE={CUSTOM_SPEECH_LOCALE}\n")

try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version,
    )

    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")
        
except (ValueError, TypeError) as e:
    print(e)

### prepare prompt to generate text


In [ ]:
import os
from azure.ai.inference.models import SystemMessage
from azure.ai.inference.models import UserMessage

NUM_SAMPLES = 5

topic = f"""
Contoso Electronics call center QnA related expected spoken utterances for {CUSTOM_SPEECH_LANG} and English languages.
"""
question = f"""
create {NUM_SAMPLES} lines of jsonl of the topic in {CUSTOM_SPEECH_LANG} and english. jsonl format is required. use 'no' as number and '{CUSTOM_SPEECH_LOCALE}', 'en-US' keys for the languages.
only include the lines as the result. Do not include ```jsonl, ``` and blank line in the result. 
"""

system_message = """
Generate plain text sentences of #topic# related text to improve the recognition of domain-specific words and phrases.
Domain-specific words can be uncommon or made-up words, but their pronunciation must be straightforward to be recognized. 
Use text data that's close to the expected spoken utterances. The nummber of utterances per line should be 1. 
Here is examples of the expected format:
{"no": 1, "string": "string", "string": "string"}
{"no": 2, "string": "string", "string": "string"}
"""

user_message = f"""
#topic#: {topic}
Question: {question}
"""

# Simple API Call
response = client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.8,
    max_tokens=1024,
    top_p=0.1
)

content = response.choices[0].message.content
print(content)
print("Usage Information:")
#print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}") #only o1 models support this
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

In [ ]:
synthetic_text_file = "cc_support_expressions.jsonl"
with open(synthetic_text_file, 'w', encoding='utf-8') as f:
    for line in content.split('\n'):
        if line.strip():  # Check if the line is not empty
            f.write(line + '\n')

%store synthetic_text_file

## 2. Create a plain text file to train custom speech models


In [ ]:
import datetime

# Check https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=stt for supported locale
#languages = ["vi-VN", "en-US"]
#languages = [CUSTOM_SPEECH_LOCALE] # List of languages to generate audio files
DELETE_OLD_DATA = True

output_dir = "plain_text"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if(DELETE_OLD_DATA):
    for file in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, file))    


with open(synthetic_text_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            expression = json.loads(line)
            for lang in languages:
                text = expression[lang]
                timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                file_name = f"{lang}_{timestamp}.txt"
                with open(os.path.join(output_dir,file_name), 'a', encoding='utf-8') as plain_text:
                    plain_text.write(f"{text}\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}")
            print(e)

In [ ]:
import glob
plain_texts = glob.glob(f'{output_dir}/*.txt')
plain_texts

for file_name in plain_texts:
    with open(file_name, 'r', encoding='utf-8') as file:
        content = file.read()
        print(f"=== File Name: {file_name} ===")
        print(content)

### Store the generated text file path in the variable `plain_text_path`


In [ ]:
plain_text_path = os.path.join(output_dir, file_name)
%store plain_text_path